<a href="https://colab.research.google.com/github/dmarinere/10academysolution/blob/master/Iyanuloluwa_scrapping_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Web scrapping using python

#### References
1. [Practical Introduction to Web Scraping in Python](https://realpython.com/python-web-scraping-practical-introduction/)
2. [Web Scraping using Python](https://www.datacamp.com/community/tutorials/web-scraping-using-python)

In [ ]:
# $ python3 -m venv venv
# $ . ./venv/bin/activate

In [ ]:
#Better
!pip install requests BeautifulSoup4 fire

     |████████████████████████████████| 81kB 3.2MB/s 
  Created wheel for fire: filename=fire-0.3.1-py2.py3-none-any.whl size=111005 sha256=b31e5f68cfa2d52db579f5254bb42599799895ae8a0b99ff3a7d36bfb0bbf422
  Stored in directory: /root/.cache/pip/wheels/c1/61/df/768b03527bf006b546dce284eb4249b185669e65afc5fbb2ac
Successfully built fire


In [ ]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
import os, sys
import re 
import fire

In [ ]:
#%%writefile ../pyscrap_url.py

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content  #.encode(BeautifulSoup.original_encoding)
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
    
def get_elements(url, tag='',search={}, fname=None):
    """
    Downloads a page specified by the url parameter
    and returns a list of strings, one per tag element
    """
    
    if isinstance(url,str):
        response = simple_get(url)
    else:
        #if already it is a loaded html page
        response = url

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        
        res = []
        if tag:    
            for li in html.select(tag):
                for name in li.text.split('\n'):
                    if len(name) > 0:
                        res.append(name.strip())
                       
                
        if search:
            soup = html            
            
            
            r = ''
            if 'find' in search.keys():
                print('findaing',search['find'])
                soup = soup.find(**search['find'])
                r = soup

                
            if 'find_all' in search.keys():
                print('findaing all of',search['find_all'])
                r = soup.find_all(**search['find_all'])
   
            if r:
                for x in list(r):
                    if len(x) > 0:
                        res.extend(x)
            
        return res

    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url))    
    
    
if get_ipython().__class__.__name__ == '__main__':
    fire(get_tag_elements)

In [ ]:
res = get_elements('https://africafreak.com/100-most-influential-twitter-users-in-africa',tag='h2')
res[:3]

['100. Jeffrey Gettleman (@gettleman)',
 '99. Africa24 Media (@a24media)',
 '98. Scapegoat (@andiMakinana)']

In [ ]:
numbers = []
for item in range(100):
  number = res[item].split('.')
  numbers.append(number)

In [ ]:
numbers[:5]

[['100', ' Jeffrey Gettleman (@gettleman)'],
 ['99', ' Africa24 Media (@a24media)'],
 ['98', ' Scapegoat (@andiMakinana)'],
 ['97', ' Africa Check (@AfricaCheck)'],
 ['96', ' James Copnall (@JamesCopnall)']]

In [ ]:
index = []
for char in numbers:
  index.append(char[0])

In [ ]:
index[:3]

['100', '99', '98']

In [ ]:
names =[]
for name in numbers:
  names.append(name[-1].split("("))


In [ ]:
names[:2]

[[' Jeffrey Gettleman ', '@gettleman)'], [' Africa24 Media ', '@a24media)']]

In [ ]:
fullname = []
for name in names:
  fullname.append(name[0].strip())


In [ ]:
fullname[:3]

['Jeffrey Gettleman', 'Africa24 Media', 'Scapegoat']

In [169]:
names[:4]

[[' Jeffrey Gettleman ', '@gettleman)'],
 [' Africa24 Media ', '@a24media)'],
 [' Scapegoat ', '@andiMakinana)'],
 [' Africa Check ', '@AfricaCheck)']]

In [150]:
handle =[]
for name in names:
  handle.append(name[-1][:-1])

In [114]:
handle[:3]


['@gettleman', '@a24media', '@andiMakinana']

In [115]:
df = pd.DataFrame(list(zip(index,fullname, handle)), columns=['Number','Name', 'Handle'])

In [116]:
df["Number"]= df.Number.astype(int)

In [117]:
df.sort_values('Number', inplace=True)

In [118]:
df.set_index('Number', inplace=True)

In [119]:
df.head()

,Name,Handle
Number,,
1,Trevor Noah,@Trevornoah
2,Gareth Cliff,@GarethCliff
3,Zuma,@SAPresident
4,News24,@News24
5,Julius Sello Malema,@Julius_S_Malema


In [ ]:
url= 'https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/#east-africa'

In [176]:
  response = simple_get(url)
  res = get_elements(response, search={'find_all':{'class_':'wp-block-embed__wrapper'}})
  pattern = "@[a-zA-Z]+"
  twitter_handle = re.findall(pattern, str(res))
  print("The total number of African leaders extracted", len(twitter_handle))
  twitter_handle[:5]

findaing all of {'class_': 'wp-block-embed__wrapper'}
The total number of African leaders extracted 52


['@EswatiniGovern',
 '@MalawiGovt',
 '@hagegeingob',
 '@FinanceSC',
 '@PresidencyZA']

## Web scrapping using bash script
If the web site has a quite simple HTML, you can easily use curl to perform the request and then extract the needed values using bash commands grep, cut , sed, ..

This tutorial is adapted from [this](https://medium.com/@LiliSousa/web-scraping-with-bash-690e4ee7f98d) medium article

In [ ]:
%%bash
curl https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/#east-africa > tmp_file


In [ ]:
%%bash 
#curl the page and save content to tmp_file
url = https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/#east-africa
curl -X GET $url -o tmp_file


#!/bin/bash

# write headers to CSV file
echo "Name, twitter_id" >> extractData.csv
n="1"
while [ $n -lt 2 ]
do
  
  #get title
  title=$(cat tmp_file | grep "class=\"css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0\" | cut -d ';' -f1 )
  
  #get auth
  twitter_id=$(cat tmp_file |grep -A1 "class=\"css-1dbjc4n r-18u37iz r-1wbh5a2"" | tail -1)

  echo "$title, $twitter_id" >> extractData.csv

  n=$[$n+1]

done

### Twitter Scraping to find the follower counts and most used Hastags

In [79]:
import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream


In [70]:
!pip install tweepy
!pip install locale
!pip install threading

ERROR: Could not find a version that satisfies the requirement locale (from versions: none)
ERROR: No matching distribution found for locale
ERROR: Could not find a version that satisfies the requirement threading (from versions: none)
ERROR: No matching distribution found for threading


In [84]:
import logging
import os

logger = logging.getLogger()

In [85]:
def create_api():
  consumer_key = os.environ.get('consumer_key')
  consumer_secret = os.environ.get('consumer_secret')
  access_token = os.environ.get('access_token')
  access_token_secret = os.environ.get('access_token_secret')
  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_token_secret)
  api = tweepy.API(auth, wait_on_rate_limit=True,
                     wait_on_rate_limit_notify=True)
  try:
    api.verify_credentials()
  except Exception as e:
    logger.error("Error creating API", exc_info=True)
    raise e
    logger.info("API created")
    return api

In [151]:
followers_count = {}
for count in range(len(handle)):
  try:
    user = api.get_user(handle[count])
    followers_count[handle[count]]= user.followers_count
  except Exception as e:
    pass

In [157]:
len(followers_count)

93

In [212]:
df['Follower_count']= df['Handle'].map(followers_count)

In [213]:
df.sort_values('Follower_count', inplace=True, ascending=False)

In [214]:
df.head()

,Name,Handle,Follower_count
Number,,,
1,Trevor Noah,@Trevornoah,10798919.0
4,News24,@News24,3573060.0
5,Julius Sello Malema,@Julius_S_Malema,3123283.0
2,Gareth Cliff,@GarethCliff,1974337.0
15,Euphonik™♛,@euphonik,1753274.0


In [215]:
first_ten = df.head(10)

In [188]:
 first_ten.to_csv ("/content/drive/My Drive/Colab Notebooks/African_influencer.csv", index = False, header=True)

In [216]:
african_follower = {}
new_one ={}
fullname = []
for count in range(len(twitter_handle)):
  try:
    user = api.get_user(twitter_handle[count])
    african_follower[twitter_handle[count]]= user.followers_count
    new_one[twitter_handle[count]]= user.name

    
  except Exception as e:
    pass

In [197]:
df2 = pd.DataFrame(twitter_handle, columns=['Twitter Handle'])

In [209]:
df2['Follower_count']= df2['Twitter Handle'].map(african_follower)

In [217]:
df2["Name"] = df2['Twitter Handle'].map(new_one)

In [219]:
df2.sort_values('Follower_count', inplace=True, ascending=False)

In [225]:
df2.drop_duplicates(inplace=True) 

In [230]:
df2.head(10)

,Twitter Handle,Follower_count,Name
16,@WHO,8090689.0,World Health Organization (WHO)
47,@MBuhari,3266027.0,Muhammadu Buhari
22,@BorisJohnson,2905525.0,Boris Johnson #StayAlert
48,@PaulKagame,1979494.0,Paul Kagame
29,@KagutaMuseveni,1807478.0,Yoweri K Museveni
4,@PresidencyZA,1596127.0,Presidency | South Africa 🇿🇦
41,@NAkufoAddo,1502376.0,Nana Akufo-Addo
21,@CyrilRamaphosa,1481545.0,Cyril Ramaphosa 🇿🇦 #StaySafe
17,@StateHouseKenya,1101220.0,State House Kenya
19,@MagufuliJP,865395.0,Dr John Magufuli


In [232]:
topten_african_leader = df.head(10)

In [233]:
topten_african_leader.to_csv("/content/drive/My Drive/Colab Notebooks/African_leaders.csv", index = False, header=True)

In [234]:
df2.head()

,Twitter Handle,Follower_count,Name
16,@WHO,8090689.0,World Health Organization (WHO)
47,@MBuhari,3266027.0,Muhammadu Buhari
22,@BorisJohnson,2905525.0,Boris Johnson #StayAlert
48,@PaulKagame,1979494.0,Paul Kagame
29,@KagutaMuseveni,1807478.0,Yoweri K Museveni
